In [1]:
import os
import cv2
import shutil
import pandas as pd
import numpy as np

from colorama import Fore, Style
from typing import List
from dotenv import load_dotenv
from DataRepository import DataRepository
from ultralytics import YOLO

load_dotenv()
STORAGE_DIR = os.getenv("STORAGE_DIR")
CROPPED_VIDEOS_FOLDER = "cropped-videos"
# IMAGES_FOLDER = "images"
# LABELS_FOLDER = "labels"
SUPPORTED_VIDEO_FORMATS = os.getenv("SUPPORTED_VIDEO_FORMATS")
SUPPORTED_IMAGE_FORMATS = os.getenv("SUPPORTED_IMAGE_FORMATS")
VIDEO_IMAGE_PREVIEW_FOLDER = os.getenv("VIDEO_IMAGE_PREVIEW_FOLDER")


In [2]:
print(STORAGE_DIR), print(CROPPED_VIDEOS_FOLDER)
# computervision/labeled_frame_nrs_to_imgs.ipynb

/media/miked/Elements/Judge/FINISHED-DB-READY
cropped-videos


(None, None)

In [3]:
os.makedirs(os.path.join(STORAGE_DIR, CROPPED_VIDEOS_FOLDER), exist_ok=True)

def get_video_path(repo, videoId):
    return os.path.join(STORAGE_DIR, repo.VideoNames.loc[videoId, "name"])

In [4]:
repo = DataRepository()

connection established, loading frame paths
rel paths of framelabels loaded


In [42]:
modeltrainround = 4
modelpath = f"/home/miked/code/judge/runs/detect/train{modeltrainround}"
modelpath

'/home/miked/code/judge/runs/detect/train4'

In [43]:
# From https://docs.ultralytics.com/modes/predict/#key-features-of-predict-mode
# Load a model
model = YOLO(os.path.join(modelpath, "weights", "best.pt"))


('/media/miked/Elements/Judge/FINISHED-DB-READY/competition/belgium/SR1/2024-sr1-xlrope-belen-pka-cam2.mp4',
 '/media/miked/Elements/Judge/FINISHED-DB-READY/cropped-videos/720_2270.mp4')

In [51]:
videoIds = [677, 1030, 1172, 1206, 1235, 1270, 1319, 1419, 1692, 1764, 1769, 1772, 1775, 1776, 1777,
            1796, 2152, 2216, 2223, 2241, 2270]

for videoId in videoIds:
    predictions_per_second = 2000
    DIM = 720
    videoPath = get_video_path(repo, videoId=videoId)
    videoOutputPath = os.path.join(STORAGE_DIR, CROPPED_VIDEOS_FOLDER, f"{DIM}_{videoId}.mp4")
    videoPath, videoOutputPath

    cap = cv2.VideoCapture(videoPath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    wait_time = int(333 / fps - 7)
    frames_to_skip = fps // predictions_per_second
    frames_to_skip
    padding_x = width // 60
    padding_y = height // 50
    N = 4
    avgIOUlastNseconds = 0.0

    smootval = 0.85
    smootval_shrink = 0.945
    smooted_x1_min = None
    smooted_y1_min = None
    smooted_x2_max = None
    smooted_y2_max = None

    max_w = 0
    max_h = 0
    min_w = width
    min_h = height
    max_wh = max(width, height)

    i = 0
    times_with_no_jumper = 0

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    out = cv2.VideoWriter(videoOutputPath, fourcc, fps, (DIM, DIM))
    ret, frame = cap.read()
    while ret:
        i += 1
        result = model(frame, verbose=False)
        xyxy_boxes = result[0].boxes.xyxy
        if xyxy_boxes.shape[0] > 0:
            x1_min = max(0, int(xyxy_boxes[:, 0].min().item()) - padding_x)
            y1_min = max(0, int(xyxy_boxes[:, 1].min().item()) - padding_y)
            x2_max = min(width, int(xyxy_boxes[:, 2].max().item()) + padding_x)
            y2_max = min(height, int(xyxy_boxes[:, 3].max().item()) + padding_y)
            times_with_no_jumper = 0
        elif times_with_no_jumper < 8:
            times_with_no_jumper += 1
        else:
            x1_min = 0
            y1_min = 0
            x2_max = width
            y2_max = height
            times_with_no_jumper += 1

        if smooted_x1_min is None:
            smooted_x1_min = x1_min
            smooted_y1_min = y1_min
            smooted_x2_max = x2_max
            smooted_y2_max = y2_max
        else:
            intersection_width = max(0, min(x2_max, smooted_x2_max) - max(x1_min, smooted_x1_min))
            intersection_height = max(0, min(y2_max, smooted_y2_max) - max(y1_min, smooted_y1_min))
            intersection_area = intersection_width * intersection_height
            area_new_box = (x2_max - x1_min) * (y2_max - y1_min)
            area_smooted_box = (smooted_x2_max - smooted_x1_min) * (smooted_y2_max - smooted_y1_min)
            union = area_smooted_box + area_new_box - intersection_area
            iou = intersection_area / union
            avgIOUlastNseconds = ((7 * fps - 1) * avgIOUlastNseconds + iou) / 7 / fps 
            avgSqrd = avgIOUlastNseconds*avgIOUlastNseconds
            iou_threshold = avgSqrd * avgSqrd
            # print(i, iou, iou_threshold)
            if iou > iou_threshold:
                smooted_x1_min = int(smootval * smooted_x1_min + (1-smootval) * x1_min) if x1_min < smooted_x1_min else int(smootval_shrink * smooted_x1_min + (1-smootval_shrink) * x1_min)
                smooted_y1_min = int(smootval * smooted_y1_min + (1-smootval) * y1_min) if y1_min < smooted_x1_min else int(smootval_shrink * smooted_y1_min + (1-smootval_shrink) * y1_min)
                smooted_x2_max = int(smootval * smooted_x2_max + (1-smootval) * x2_max) if x2_max > smooted_x2_max else int(smootval_shrink * smooted_x2_max + (1-smootval_shrink) * x2_max)
                smooted_y2_max = int(smootval * smooted_y2_max + (1-smootval) * y2_max) if y2_max > smooted_y2_max else int(smootval_shrink * smooted_y2_max + (1-smootval_shrink) * y2_max)

        w_jumpers = smooted_x2_max - smooted_x1_min
        h_jumpers = smooted_y2_max - smooted_y1_min
        max_w = max(max_w, w_jumpers)
        max_h = max(max_h, h_jumpers)
        min_w = min(min_w, w_jumpers)
        min_h = min(max_h, h_jumpers)

        cropped_frame = frame[smooted_y1_min:smooted_y2_max, smooted_x1_min:smooted_x2_max]
        max_wh_jumpers = max(w_jumpers, h_jumpers)
        zeros = np.zeros((max_wh_jumpers, max_wh_jumpers, 3), dtype=np.uint8)
        offset_x = (max_wh_jumpers - w_jumpers) // 2
        offset_y = (max_wh_jumpers - h_jumpers) // 2
        zeros[offset_y:offset_y+h_jumpers, offset_x:offset_x+w_jumpers] = cropped_frame
        zeros = cv2.resize(zeros, (DIM, DIM))

        out.write(zeros)
        
        # cv2.imshow('Video', zeros)

        # # Skip frames
        # s = frames_to_skip
        # while ret and s > 0:
        #     ret, _ = cap.read()
        #     s -= 1
        
        # if cv2.waitKey(wait_time) & 0xFF == ord('q'):
        #     break
        ret, frame = cap.read()

    print("min width", min_w, "max_w", max_w)
    print("min_height", min_h, "max_h", max_h)
    print(i)

    # Release everything if job is finished
    out.release()
    cap.release()
    cv2.destroyAllWindows()

min width 167 max_w 620
min_height 248 max_h 323
2486
min width 172 max_w 1840
min_height 972 max_h 1074
4742
min width 398 max_w 674
min_height 363 max_h 410
2072
min width 163 max_w 506
min_height 191 max_h 227
2325
min width 237 max_w 714
min_height 180 max_h 376
1925
min width 236 max_w 576
min_height 429 max_h 435
2336
min width 209 max_w 654
min_height 240 max_h 290
1670
min width 103 max_w 1907
min_height 120 max_h 1072
7765
min width 117 max_w 303
min_height 209 max_h 324
3176
min width 180 max_w 578
min_height 262 max_h 668
4027
min width 208 max_w 869
min_height 277 max_h 857
3772
min width 122 max_w 293
min_height 119 max_h 432
4156
min width 181 max_w 589
min_height 374 max_h 811
3730
min width 112 max_w 1914
min_height 893 max_h 1074
6396
min width 120 max_w 254
min_height 213 max_h 341
3817
min width 378 max_w 1053
min_height 304 max_h 482
2374
min width 299 max_w 1095
min_height 494 max_h 590
4848
min width 118 max_w 254
min_height 149 max_h 270
3913
min width 118 max_w 